In [1]:
from sqlalchemy import create_engine, text
import numpy as np
import pandas as pd

import credentials

In [2]:
output_file = "helechos_vel_group_aff_2025-11.csv"

In [3]:
conn_str = 'mysql+mysqlconnector://' + \
	f"{credentials.mysql['username']}:{credentials.mysql['password']}" + \
	'@localhost:3306/Mutis'
engine = create_engine(conn_str)
connection = engine.connect()

In [4]:
ids = pd.read_sql_table("Identifications", connection)

/home/nelson/anaconda3/lib/python3.13/site-packages/pandas/io/sql.py:1725: SAWarning: Did not recognize type 'polygon' of column 'Shape'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)


In [5]:
noiddate = pd.read_sql(
	"select IdentificationID,DateEnd from Identifications left join Occurrences on Occurrence=OccurrenceID where Date is NULL",
	connection)

In [6]:
ids = ids.merge(noiddate, how='left', on='IdentificationID')

In [7]:
ids.loc[ids.Date.isnull(), 'Date'] = pd.to_datetime(ids.loc[ids.Date.isnull(), 'DateEnd'])

In [9]:
ids.head()

,IdentificationID,Occurrence,Name,HybridSecondaryParental,Certainty,IdentifiedByVerbatim,IdentifiedBy,Publication,Date,TimeStamp,DateEnd
0,1,1,756,NaN,None,None,None,None,1999-02-10,2025-06-18 09:21:51,1999-02-10
1,2,2,869,NaN,None,"Vargas, C. A.",None,None,2016-08-01,2025-06-18 09:21:51,NaN
2,3,3,2271,NaN,None,"Urbano, S.",None,None,2018-04-01,2025-06-18 09:21:51,NaN
3,4,4,3313,NaN,None,"Orejuela, Andrés",None,None,2013-08-01,2025-06-18 09:21:51,NaN
4,5,5,3497,NaN,cf.,"Cabrera, Diego",None,None,2018-09-01,2025-06-18 09:21:51,NaN


In [ ]:
# Debugging selecting last ided occurrences
#dt = pd.DataFrame.from_dict({
#	'd':['2014-10-16', '2014-10-18', '2014-10-16', '2016-10-16', None, None],
#	'v':[1,1,2,2,2,3],
#	'a':[3,4,4,3,3,4],
#	'i': [1,2,3,4,5,6]})
#dt['d'] = pd.to_datetime(dt.d)
#l = dt.groupby(['v']).d.idxmax()
#onlynas = dt.loc[dt.v.isin(l[l.isnull()].index)].groupby('v').first().reset_index()[dt.columns] # Only NaT dates
#pd.concat([onlynas, dt.loc[l[l.notna()]]])

,d,v,a,i
0,NaT,3,4,6
1,2014-10-18,1,4,2
3,2016-10-16,2,3,4


In [11]:
lastids = ids.groupby('Occurrence').Date.idxmax()

In [12]:
onlynas = ids.loc[
		ids.Occurrence.isin(lastids[lastids.isnull()].index)
	].groupby('Occurrence'
	).first(
	).reset_index(
	)[ids.columns]

In [13]:
id_pool = pd.concat([onlynas, ids.loc[lastids[lastids.notnull()]]]).IdentificationID

In [14]:
tax = pd.read_sql_table("Taxa", connection)

In [15]:
clades = [
	'Polypodiidae',
	'Lycopodiopsida',
	'Ophioglossidae',
	'Equisetidae',
	'Marattiidae'
]

In [16]:
target = []
papa = tax.loc[tax.Name.isin(clades), 'TaxonID'].tolist()

while papa:
	target += papa
	papa = tax.loc[tax.Parent.isin(papa), 'TaxonID'].tolist()
	#print(papa)

In [17]:
pooltax = ', '.join([str(i) for i in target])

In [18]:
tax[tax.TaxonID.isin(target)].head()

,TaxonID,Name,AcceptedName,Authority,Author,Protologue,Parent,Comment,CheckPriority,Distribution,TimeStamp
65,66,Acrostichum aureum,None,None,L.,None,4002.0,None,NaN,None,2025-06-18 09:14:24
66,67,Adiantum,None,None,None,None,2934.0,None,NaN,None,2025-06-18 09:14:24
67,68,Adiantum concinnum,None,None,Humb. & Bonpl. ex Willd.,None,67.0,None,NaN,None,2025-06-18 09:14:24
68,69,Adiantum patens,None,None,Willd.,None,67.0,None,NaN,None,2025-06-18 09:14:24
69,70,Adiantum petiolatum,None,None,Desv.,None,67.0,None,NaN,None,2025-06-18 09:14:24


In [19]:
poollist = ', '.join([str(x) for x in id_pool.tolist()])

In [ ]:
recs = pd.read_sql(
	"SELECT Specimens.SpecimenCode AS 'catalogNumber', Occurrences.Type AS 'basisOfRecord', Specimens.Institution AS 'institutionCode', Sources.Author AS 'refAuthor', Sources.Name AS 'refName', Sources.Year AS 'refYear', Occurrences.CollectorVerbatim AS 'recordedBy', Occurrences.CollectionNumberVerbatim AS 'recordNumber', Occurrences.DateInit AS 'eventDate', Taxa.Name AS 'scientificName', Taxa.Author AS 'scientificNameAuthorship', Identifications.IdentifiedByVerbatim AS 'identifiedBy', Identifications.Date AS 'dateIdentified', Locations.Country AS 'country', Locations.Admin01 AS 'stateProvince', Locations.Admin02 AS 'municipality', Locations.Admin03 as 'county', Locations.Name AS 'localityVerbatim', Geocodings.InterpretedLat AS 'decimalLatitude', Geocodings.InterpretedLon AS 'decimalLongitude', Locations.ElevationMin AS 'minimumElevationInMeters', Locations.ElevationMax AS 'maximumElevationInMeters' FROM Occurrences " \
	+ "LEFT JOIN Sources ON SourceID=Occurrences.Reference " \
	+ "LEFT JOIN Identifications ON Identifications.Occurrence=OccurrenceID " \
	+ "LEFT JOIN Taxa ON TaxonID=Identifications.Name " \
	+ "LEFT JOIN Specimens ON Specimens.Occurrence=OccurrenceID " \
	+ "LEFT JOIN Locations ON LocationID=Occurrences.Location " \
	+ "LEFT JOIN Geocodings ON LocationID=Geocodings.Location " \
	+ f"WHERE Identifications.IdentificationID IN ({poollist}) " \
	+ f"AND Taxa.TaxonID IN ({pooltax})",
	connection)

In [21]:
recs['taxonRank'] = None

In [22]:
recs.loc[recs.scientificName.str.contains(' var. ', regex=False), 'taxonRank'] = 'variety'

In [23]:
recs.loc[recs.scientificName.str.contains(' subsp. ', regex=False), 'taxonRank'] = 'subspecies'

In [24]:
recs.loc[recs.scientificName.str.contains(' fo?\\. ', regex=True), 'taxonRank'] = 'forma'

In [25]:
if recs.loc[recs.scientificName.str.contains('ales$', na=False)
	].shape[0] > 0:

	recs.loc[
		recs.scientificName.str.contains('ales$', na=False),
		'taxonRank'] = 'order'

In [26]:
if recs.loc[recs.scientificName.str.contains('aceae$', na=False)
	].shape[0] > 0:

	recs.loc[
		recs.scientificName.str.contains('aceae$', na=False),
		'taxonRank'] = 'family'

In [27]:
if recs.loc[recs.scientificName.str.contains('×', na=False)
	].shape[0] > 0:

	recs.loc[
		recs.scientificName.str.contains('×', na=False),
		'taxonRank'] = 'hybrid'

In [28]:
recs.loc[
	recs.taxonRank.isnull() &
	recs.scientificName.str.contains('[\\w\\-]\\s+[\\w\\-]', regex=True),
	'taxonRank'] = 'species'

In [29]:
# everything else is ided to genus level
if recs.loc[recs.taxonRank.isnull()].shape[0] > 0:
	recs.loc[recs.taxonRank.isnull(), 'taxonRank'] = 'genus'

In [30]:
recs['genus'] = None
recs['specificEpithet'] = None
recs['infraspecificEpithet'] = None

In [31]:
recs.loc[recs.taxonRank == 'species', 'specificEpithet'
	] = recs.loc[recs.taxonRank == 'species', 'scientificName'
	].str.extract('\\s+([\\w\\-]+)$')[0]

In [32]:
recs.loc[recs.taxonRank.isin(['variety', 'forma', 'subspecies']), 'specificEpithet'
	] = recs.loc[recs.taxonRank.isin(['variety', 'forma', 'subspecies']), 'scientificName'
	].str.extract('\\s+([\\w\\-]+)\\s+[\\w\\-]+$')

In [33]:
recs.loc[recs.taxonRank.isin(['variety', 'forma', 'subspecies']), 'infraspecificEpithet'
	] = recs.loc[recs.taxonRank.isin(['variety', 'forma', 'subspecies']), 'scientificName'
	].str.extract('\\s+[\\w\\-]+\\s+([\\w\\-]+)$')

In [34]:
recs.loc[recs.taxonRank == 'genus', 'genus'
	] = recs.loc[recs.taxonRank == 'genus', 'scientificName']

In [35]:
recs.loc[
	recs.taxonRank.isin(['species', 'hybrid', 'variety', 'forma', 'subspecies']),
	'genus'
	] = recs.loc[
	recs.taxonRank.isin(['species', 'hybrid', 'variety', 'forma', 'subspecies'])
	].scientificName.str.replace('×', ' '
	).str.replace('^\\s+|\\s+$', '', regex=True
	).str.replace('\\s+', ' ', regex=True
	).str.split('\\s+', expand=True
	)[0]

In [36]:
def get_family(row):
	if row.taxonRank == 'family':
		return row.scientificName
	elif row.genus:
		thpapa = tax.loc[tax.Name == row.genus, 'Parent'].item()
		return tax.loc[tax.TaxonID == thpapa, 'Name'].item()
	else:
		return None


In [37]:
recs['family'] = recs.apply(get_family, axis=1)

In [38]:
recs.columns

Index(['catalogNumber', 'basisOfRecord', 'institutionCode', 'refAuthor',
       'refName', 'refYear', 'refJournal', 'recordedBy', 'recordNumber',
       'eventDate', 'scientificName', 'scientificNameAuthorship',
       'identifiedBy', 'dateIdentified', 'country', 'stateProvince',
       'municipality', 'county', 'localityVerbatim', 'decimalLatitude',
       'decimalLongitude', 'minimumElevationInMeters',
       'maximumElevationInMeters', 'taxonRank', 'genus', 'specificEpithet',
       'infraspecificEpithet', 'family'],
      dtype='object')

In [121]:
authlist = ', '.join([
	str(x) for x in recs.groupby('refAuthor'
		).size(
		).reset_index(
		).refAuthor.tolist()
])

In [122]:
authors = pd.read_sql(
	"SELECT People, LastName, `Order` FROM PeoplePersons " \
	+ "LEFT JOIN Persons ON PeoplePersons.Person=Persons.PersonID " \
	+ f"WHERE PeoplePersons.People IN ({authlist})",
	connection)

In [124]:
authors = authors.sort_values(['People', 'Order'])

In [125]:
cita = authors.groupby('People').size().reset_index()

In [126]:
cita['author'] = None

In [127]:
for a in cita.People:
	#print(f"{a=}")
	l = authors.loc[authors.People == a, 'LastName'].values.tolist()
	l = [m for m in l if pd.notnull(m)]
	#print(f"{l=}")
	if len(l) == 1:
		cita.loc[cita.People == a, 'author'] = l[0]
	elif len(l) == 2:
		cita.loc[cita.People == a, 'author'] = l[0] + ' & ' + l[1]
	elif len(l) > 2:
		acc = ', '.join(l[:-1]) + ' & ' + l[-1]
		cita.loc[cita.People == a, 'author'] = acc
	 


In [130]:
recs = recs.merge(cita, how='left', left_on='refAuthor', right_on='People')

In [133]:
def get_citation(row):
	out = None
	if row.refYear and row.author and row.refName:
		out = f"{row.author}. {int(row.refYear)}. {row.refName}"
	return out

recs['bibliographicCitation'] = recs.apply(get_citation, axis=1) 

In [136]:
recs = recs[
	['catalogNumber', 'basisOfRecord', 'institutionCode',
	'bibliographicCitation',
	'recordedBy', 'recordNumber', 'eventDate', 
	'taxonRank', 'scientificName', 'scientificNameAuthorship', 
	'family', 'genus', 'specificEpithet', 'infraspecificEpithet',
	'identifiedBy', 'dateIdentified', 
	'country', 'stateProvince', 'municipality', 'county', 'localityVerbatim',
	'decimalLatitude', 'decimalLongitude', 
	'minimumElevationInMeters', 'maximumElevationInMeters']
]

In [137]:
recs.to_csv(output_file, index=False)

In [55]:
connection.close()